In [1]:
!pip install transformers datasets huggingface-hub >> /dev/null
!git clone https://huggingface.co/spaces/briaai/BRIA-RMBG-1.4/ >> /dev/null
%cd BRIA-RMBG-1.4

Cloning into 'BRIA-RMBG-1.4'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 190 (delta 3), reused 0 (delta 0), pack-reused 180
Receiving objects: 100% (190/190), 383.84 KiB | 25.59 MiB/s, done.
Resolving deltas: 100% (108/108), done.
/kaggle/working/BRIA-RMBG-1.4


In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from torchvision.transforms.functional import normalize
from briarmbg import BriaRMBG
import PIL
from PIL import Image
from typing import Tuple


net = BriaRMBG.from_pretrained("briaai/RMBG-1.4")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net.to(device)

    
def resize_image(image):
    image = image.convert('RGB')
    model_input_size = (1024, 1024)
    image = image.resize(model_input_size, Image.BILINEAR)
    return image


def process(image):

    # prepare input
    orig_image = Image.fromarray(image)
    w,h = orig_im_size = orig_image.size
    image = resize_image(orig_image)
    im_np = np.array(image)
    im_tensor = torch.tensor(im_np, dtype=torch.float32).permute(2,0,1)
    im_tensor = torch.unsqueeze(im_tensor,0)
    im_tensor = torch.divide(im_tensor,255.0)
    im_tensor = normalize(im_tensor,[0.5,0.5,0.5],[1.0,1.0,1.0])
    if torch.cuda.is_available():
        im_tensor=im_tensor.cuda()

    #inference
    result=net(im_tensor)
    # post process
    result = torch.squeeze(F.interpolate(result[0][0], size=(h,w), mode='bilinear') ,0)
    ma = torch.max(result)
    mi = torch.min(result)
    result = (result-mi)/(ma-mi)    
    # image to pil
    im_array = (result*255).cpu().data.numpy().astype(np.uint8)
    pil_im = Image.fromarray(np.squeeze(im_array))
    # paste the mask on the original image
    new_im = Image.new("RGBA", pil_im.size, (0,0,0,0))
    new_im.paste(orig_image, mask=pil_im)
    # new_orig_image = orig_image.convert('RGBA')

    return new_im

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/177M [00:00<?, ?B/s]

In [3]:
def to_rgb(rgba):
    background = Image.new('RGBA', rgba.size, (255,255,255))
    return Image.alpha_composite(background, rgba).convert('RGB')

In [4]:
synonymous_sentences = [
    "Convert the picture to a decal.",
    "Transform the photo into a sticker.",
    "Change the image into a sticker form.",
    "Make the image into a decal.",
    "Render the picture as a sticker.",
    "Process the image to become a sticker.",
    "Adapt the photo into a sticker format.",
    "Modify the image to be a sticker.",
    "Evolvethe picture into a sticker.",
    "Shift the image into sticker mode.",
    "Turn the visual into a sticker.",
    "Update the image to sticker status.",
    "Refashion the photo into a sticker.",
    "Reimagine the image as a sticker.",
    "Convert the snapshot into a sticker.",
    "Morph the picture into a sticker design.",
    "Change this image into a sticker version.",
    "Make this picture a sticker.",
    "Turn the photograph into a sticker.",
    "Shape the image into a sticker.",
    "Transform this visual representation into a sticker.",
    "Fashion the image into a sticker.",
    "Alter the photo to sticker form.",
    "Remake the image into a sticker.",
    "Craft the picture into a sticker.",
    "Develop the image into a sticker.",
    "Turn the illustration into a sticker.",
    "Switch the image to a sticker.",
    "Recreate the photo as a sticker.",
    "Reform the image into a sticker.",
    "Revamp the picture to be a sticker.",
    "Refine the image into a sticker.",
    "Redesign the photo into a sticker.",
    "Translate the image to a sticker.",
    "Convert the figure into a sticker.",
    "Make the visual graphic into a sticker.",
    "Adapt the snapshot to a sticker.",
    "Mold the image into a sticker.",
    "Transform the imagery into a sticker.",
    "Modify this visual into a sticker.",
    "Convert the representation into a sticker.",
    "Change the graphic into a sticker.",
    "Process the photograph into a sticker.",
    "Adapt the picture to become a sticker.",
    "Morph the visual into a sticker.",
    "Transform the scene into a sticker.",
    "Turn the visual image into a sticker."
]
synonymous_sentences = [i for i in synonymous_sentences if 'sticker' in i]
len(synonymous_sentences)

45

In [5]:
from datasets import load_dataset

dataset = load_dataset("misshimichka/flower_faces_dataset_v3")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/misshimichka--flower_faces_dataset_v3-f811bd0156300ee2/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
clear_dataset = {
    'original_image': [],
    'edit_prompt': [], 
    'cartoonized_image': []
}

In [ ]:
from datasets import Dataset
import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline


for item in dataset["train"]:        
    no_bg_cartoonized = to_rgb(process(np.asarray(item["cartoonized_image"])))
        
    f, axarr = plt.subplots(1, 2, figsize=(7, 7))
    axarr[0].set_xlabel(item["edit_prompt"])
    axarr[0].imshow(item["original_image"])
    axarr[1].imshow(no_bg_cartoonized)
    plt.show()
    
    answer = input()
    if answer == "ok":
        clear_dataset['original_image'].append(item["original_image"])
        clear_dataset['edit_prompt'].append(item["edit_prompt"])
        clear_dataset['cartoonized_image'].append(no_bg_cartoonized)

In [10]:
clear_dataset = Dataset.from_dict(clear_dataset)
clear_dataset.save_to_disk("/kaggle/working/result")

In [11]:
%cd /kaggle/working
!zip result.zip -r /kaggle/working/result

/kaggle/working
  adding: kaggle/working/result/ (stored 0%)
  adding: kaggle/working/result/state.json (deflated 40%)
  adding: kaggle/working/result/dataset.arrow (deflated 0%)
  adding: kaggle/working/result/dataset_info.json (deflated 63%)


In [12]:
from IPython.display import FileLink
FileLink(r'result.zip')

/kaggle/working/result.zip